## Nepal Breakdown

In [64]:
import pandas as pd
df = pd.read_csv('C:/Users/tkmwangi/Documents/Data/COMBINED_SICS_RMS_DMP_ORG.csv')
df['CEDENT_COUNTRY'].value_counts()
nepal_only = df.loc[df['CEDENT_COUNTRY'] == 'NEPAL']
#Filtering to only have post 2015 data
nepal_only = nepal_only.loc[nepal_only['UNDERWRITING_YEAR'] >= 2015]
#nepal_only.info()
#Replacing repeated cedant names that are duplicated
nepal_only['CEDENT_NAME'] = nepal_only['CEDENT_NAME'].replace('GENERAL INSURANCE COMPANY LIMITED NEPAL', 'GENERAL INSURANCE COMPANY LIMITED, NEPAL')
nepal_only['CEDENT_NAME'] = nepal_only['CEDENT_NAME'].replace('AJOD INSURANCE LIMITED NEPAL', 'AJOD INSURANCE LIMITED, NEPAL')
nepal_only['CEDENT_NAME'] = nepal_only['CEDENT_NAME'].replace('IME GENERAL INSURANCE COMPANYNEPAL', 'IME GENERAL INSURANCE COMPANY,NEPAL')
nepal_only['CEDENT_NAME'] = nepal_only['CEDENT_NAME'].replace('PRABHU INSURANCE LIMITEDNEPAL','PRABHU INSURANCE LIMITED,NEPAL')

C:\Users\tkmwangi\AppData\Local\Temp\ipykernel_23160\3228248182.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/tkmwangi/Documents/Data/COMBINED_SICS_RMS_DMP_ORG.csv')


##  Summary Overview

* There are number 14 total cedants from nepal within the time period, 2015 - 2021
* 2018 saw the largest jump un number of cedants from 8 to 14 in a single year, prior to 2018 the number of cedants was 8.
* Fire is the most common line of bussiness associated with CAT, followed by Whole account and Motor 

### Let's now get into the details, the plan is to find out which cedants/business is profitable or not

1. Well split it by year and focus on the profitability by cedant for each year from 2015 to present
2. Then split it by cedant and Line of business

* We will also create a inflow and outflow column to be used later on

In [65]:
nepal_only['Inflows'] = nepal_only['BOOKED_PREMIUM'] + nepal_only['INTEREST_ON_DEPOSITS'] + nepal_only['PREMIUM_PORTFORLIO_IN'] 
nepal_only['Outflows'] = nepal_only['COMMISSION'] + nepal_only['BROKERAGE'] +nepal_only['PROFIT_COMMISSION'] + nepal_only['TAX_ON_INTEREST']  + nepal_only['OTHER_CHARGES'] + nepal_only['OVERRIDER_COMMISSION'] + nepal_only['CLAIM_PAID'] + nepal_only['PREMIUM_PORTFORLIO_OUT'] + nepal_only['OUTSTANDING_LOSSES'] + nepal_only['MANAGEMENT_EXPENSES']
nepal_only['Diff'] = nepal_only['Inflows'] - nepal_only['Outflows']
# Dropping the columns to make it easier to keep track of
columns_to_drop = ['BOOKED_PREMIUM', 'COMMISSION','INTEREST_ON_DEPOSITS','BROKERAGE','PREMIUM_PORTFORLIO_IN','PROFIT_COMMISSION','TAX_ON_INTEREST','OTHER_CHARGES','OVERRIDER_COMMISSION','CLAIM_PAID','PREMIUM_PORTFORLIO_OUT','OUTSTANDING_LOSSES','MANAGEMENT_EXPENSES']
nepal_only = nepal_only.drop(columns_to_drop, axis=1)
#sn.kdeplot(x = nepal_only['Diff'], hue=nepal_only['UNDERWRITING_YEAR'], palette="crest")
print('\n The graph indicates that the distribution of diff values has been quite similar over the period in question. \n With most values being near zero, with the negative values having a large range than the positive values')
max_diff = nepal_only['Diff'].max()
min_diff = nepal_only['Diff'].min()
print('\n The largest positive value is ', max_diff, ' with the largest negative value being ',min_diff)

ds = nepal_only['Diff'].describe()
print('\n \n The statistical analysis of the difference in inflows vs outflows is as follows',ds)


 The graph indicates that the distribution of diff values has been quite similar over the period in question. 
 With most values being near zero, with the negative values having a large range than the positive values

 The largest positive value is  6749549.76  with the largest negative value being  -15478234.0

 
 The statistical analysis of the difference in inflows vs outflows is as follows count    9.929000e+03
mean     3.400547e+04
std      4.649218e+05
min     -1.547823e+07
25%     -5.696000e+03
50%      1.508370e+03
75%      8.425996e+04
max      6.749550e+06
Name: Diff, dtype: float64


### Lets begin drilling down year by year

In [66]:
# Create an empty dictionary to store the DataFrames
nepal_years = {}

# Iterate over unique years in the 'UNDERWRITING_YEAR' column
for year in nepal_only['UNDERWRITING_YEAR'].unique():
    # Filter the DataFrame for the current year
    year_df = nepal_only.loc[nepal_only['UNDERWRITING_YEAR'] == year].copy()
    # Add the DataFrame to the dictionary with the year as the key
    nepal_years[year] = year_df

# Access the DataFrames for each year
nepal_2015 = nepal_years[2015]
nepal_2016 = nepal_years[2016]
nepal_2017 = nepal_years[2017]
nepal_2018 = nepal_years[2018]
nepal_2019 = nepal_years[2019]
nepal_2020 = nepal_years[2020]
nepal_2021 = nepal_years[2021]
nepal_2022 = nepal_years[2022]

### 2015

In [67]:
print('There are 116 unique contracts in 2015, from 8 cedants')
print('\n These are:', '\n\n\n The numbers on the right indicate the number of times the cedant name appears in the table')
current = nepal_2015[['CONTRACT_NO','CEDENT_NAME']].drop_duplicates()
current['CEDENT_NAME'].value_counts()


There are 116 unique contracts in 2015, from 8 cedants

 These are: 


 The numbers on the right indicate the number of times the cedant name appears in the table


CEDENT_NAME
PRUDENTIAL INSURANCE CO LTD            24
NECO INSURANCE CO. LTD                 21
SIDDHARTHA INSURANCE CO. LTD. NEPAL    17
RASTRIYA BEEMA SANSTHAN                16
EVEREST INSURANCE CO.LTD               15
NEPAL INSURANCE LTD.                   14
PREMIER INSURANCE CO.(NEPAL) LTD.       8
NLG INSURANCE COMPANY LIMITED           1
Name: count, dtype: int64

In [68]:
x = filtered_nepal_2021['ACT_LOB'].value_counts()
print(' The frequency of each LOB is as follows ', x)

 The frequency of each LOB is as follows  ACT_LOB
FIRE             62
WHOLE_ACCOUNT    50
MOTOR             8
Name: count, dtype: int64


In [69]:
# Create a list of values to filter, specifically anything related to CAT 
remove_list = ['XL-CAT THIRD','XL-CAT SECOND','XL-CAT FIRST','XL-CAT FOURTH','XL-CAT FIFTH','XL-RISK/CAT FOURTH','XL-RISK/CAT FIFTH','XL-CAT SIXTH']

# Remove rows based on the list of values
filtered_nepal_2015 = nepal_2015[nepal_2015['TYPE_OF_PARTICIPATION'].isin(remove_list)]
data = filtered_nepal_2015['TYPE_OF_PARTICIPATION']


filtered_nepal_2015['Diff'].value_counts()
count_negative = (filtered_nepal_2015['Diff'] < 0).sum()
count_positive = (filtered_nepal_2015['Diff'] > 0).sum()
count_zero = (filtered_nepal_2015['Diff'] == 0).sum()
print('Difference values are calculated by summing inputs and outflow and finding the difference \n')
print('The number of negative difference values is: ',count_negative, 'and the number of positive values is: ',count_positive , '.')


Difference values are calculated by summing inputs and outflow and finding the difference 

The number of negative difference values is:  0 and the number of positive values is:  69 .


### 2016

In [70]:
print('There are 123 unique contracts in 2016, from 8 cedants')
print('\n These are:')
current = nepal_2016[['CONTRACT_NO','CEDENT_NAME']].drop_duplicates()
current['CEDENT_NAME'].value_counts()

There are 123 unique contracts in 2016, from 8 cedants

 These are:


CEDENT_NAME
EVEREST INSURANCE CO.LTD               25
NECO INSURANCE CO. LTD                 22
SIDDHARTHA INSURANCE CO. LTD. NEPAL    22
PRUDENTIAL INSURANCE CO LTD            16
RASTRIYA BEEMA SANSTHAN                16
NEPAL INSURANCE LTD.                   15
PREMIER INSURANCE CO.(NEPAL) LTD.       6
NLG INSURANCE COMPANY LIMITED           1
Name: count, dtype: int64

In [71]:
# Create a list of values to filter, specifically anything related to CAT 
remove_list = ['XL-CAT THIRD','XL-CAT SECOND','XL-CAT FIRST','XL-CAT FOURTH','XL-CAT FIFTH','XL-RISK/CAT FOURTH','XL-RISK/CAT FIFTH','XL-CAT SIXTH']

# Remove rows based on the list of values
filtered_nepal_2016 = nepal_2016[nepal_2016['TYPE_OF_PARTICIPATION'].isin(remove_list)]
data = filtered_nepal_2016['TYPE_OF_PARTICIPATION']
x = filtered_nepal_2016['ACT_LOB'].value_counts()
print('The frequency of each LOB is as follows ', x)

The frequency of each LOB is as follows  ACT_LOB
FIRE             50
WHOLE_ACCOUNT    30
MARINE            6
Name: count, dtype: int64


In [72]:
filtered_nepal_2016['Diff'].value_counts()
count_negative = (filtered_nepal_2016['Diff'] < 0).sum()
count_positive = (filtered_nepal_2016['Diff'] > 0).sum()
print('The number of negative difference values is: ',count_negative, 'and the number of positive values is: ',count_positive)
print('\n The are more positives than negatives')

The number of negative difference values is:  0 and the number of positive values is:  86

 The are more positives than negatives


### 2017

In [73]:
print('There are 133 unique contracts in 2017, from 8 cedants')
print('\n These are:', '\n\n\n The numbers on the right indicate the number of times the cedant name appears in the table')
current = nepal_2017[['CONTRACT_NO','CEDENT_NAME']].drop_duplicates()
current['CEDENT_NAME'].value_counts()

There are 133 unique contracts in 2017, from 8 cedants

 These are: 


 The numbers on the right indicate the number of times the cedant name appears in the table


CEDENT_NAME
EVEREST INSURANCE CO.LTD               27
SIDDHARTHA INSURANCE CO. LTD. NEPAL    22
NECO INSURANCE CO. LTD                 21
PRUDENTIAL INSURANCE CO LTD            18
RASTRIYA BEEMA SANSTHAN                17
NEPAL INSURANCE LTD.                   16
PREMIER INSURANCE CO.(NEPAL) LTD.       8
HIMALAYAN GENERAL INSURANCE CO LTD      4
Name: count, dtype: int64

In [74]:
# Create a list of values to filter, specifically anything related to CAT 
remove_list = ['XL-CAT THIRD','XL-CAT SECOND','XL-CAT FIRST','XL-CAT FOURTH','XL-CAT FIFTH','XL-RISK/CAT FOURTH','XL-RISK/CAT FIFTH','XL-CAT SIXTH']

# Remove rows based on the list of values
filtered_nepal_2017 = nepal_2017[nepal_2017['TYPE_OF_PARTICIPATION'].isin(remove_list)]
data = filtered_nepal_2017['TYPE_OF_PARTICIPATION']

x = filtered_nepal_2017['ACT_LOB'].value_counts()
print('The frequency of each LOB is as follows ', x, '\n\n')


filtered_nepal_2017['Diff'].value_counts()
count_negative = (filtered_nepal_2017['Diff'] < 0).sum()
count_positive = (filtered_nepal_2017['Diff'] > 0).sum()
print('The number of negative difference values is: ',count_negative, 'and the number of positive values is: ',count_positive)
print('\n The are more negatives than positives')

The frequency of each LOB is as follows  ACT_LOB
FIRE             52
WHOLE_ACCOUNT    36
Name: count, dtype: int64 


The number of negative difference values is:  0 and the number of positive values is:  88

 The are more negatives than positives


### 2018

In [75]:
print('There are 255 unique contracts in 2018, from 17 cedants')
print('\n These are:','\n\n\n *The numbers on the right indicate the number of times the cedant name appears in the table')
current = nepal_2018[['CONTRACT_NO','CEDENT_NAME']].drop_duplicates()
current['CEDENT_NAME'].value_counts()

There are 255 unique contracts in 2018, from 17 cedants

 These are: 


 *The numbers on the right indicate the number of times the cedant name appears in the table


CEDENT_NAME
PRUDENTIAL INSURANCE CO LTD                 27
NECO INSURANCE CO. LTD                      25
HIMALAYAN GENERAL INSURANCE CO LTD          25
SIDDHARTHA INSURANCE CO. LTD. NEPAL         24
SANIMA INSURANCE COMPANY LIMITED            20
PRABHU INSURANCE LIMITED,NEPAL              19
RASTRIYA BEEMA SANSTHAN                     18
AJOD INSURANCE LIMITED, NEPAL               18
GENERAL INSURANCE COMPANY LIMITED, NEPAL    18
IME GENERAL INSURANCE COMPANY,NEPAL         17
EVEREST INSURANCE CO.LTD                    17
NEPAL INSURANCE LTD.                        16
PREMIER INSURANCE CO.(NEPAL) LTD.           11
Name: count, dtype: int64

In [76]:
# Create a list of values to filter, specifically anything related to CAT 
remove_list = ['XL-CAT THIRD','XL-CAT SECOND','XL-CAT FIRST','XL-CAT FOURTH','XL-CAT FIFTH','XL-RISK/CAT FOURTH','XL-RISK/CAT FIFTH','XL-CAT SIXTH']

# Remove rows based on the list of values
filtered_nepal_2018 = nepal_2018[nepal_2018['TYPE_OF_PARTICIPATION'].isin(remove_list)]
data = filtered_nepal_2018['TYPE_OF_PARTICIPATION']
x = filtered_nepal_2018['ACT_LOB'].value_counts()
print('The frequency of each LOB is as follows ', x)

The frequency of each LOB is as follows  ACT_LOB
FIRE             90
WHOLE_ACCOUNT    65
Name: count, dtype: int64


In [77]:
filtered_nepal_2018['Diff'].value_counts()
count_negative = (filtered_nepal_2018['Diff'] < 0).sum()
count_positive = (filtered_nepal_2018['Diff'] > 0).sum()
print('The number of negative difference values is: ',count_negative, 'and the number of positive values is: ',count_positive)
print('\n The are more positives than negatives')

The number of negative difference values is:  12 and the number of positive values is:  121

 The are more positives than negatives


### 2019

In [78]:
#CONTRACT_NO CEDENT_NAME
print('There are 272 unique contracts in 2019, from 17 cedants')
print('\n These are:', '\n\n\n The numbers on the right indicate the number of times the cedant name appears in the table')
current = nepal_2019[['CONTRACT_NO','CEDENT_NAME']].drop_duplicates()
current['CEDENT_NAME'].value_counts()

There are 272 unique contracts in 2019, from 17 cedants

 These are: 


 The numbers on the right indicate the number of times the cedant name appears in the table


CEDENT_NAME
NECO INSURANCE CO. LTD                      26
SIDDHARTHA INSURANCE CO. LTD. NEPAL         24
HIMALAYAN GENERAL INSURANCE CO LTD          24
PRUDENTIAL INSURANCE CO LTD                 24
PRABHU INSURANCE LIMITED,NEPAL              23
NEPAL INSURANCE LTD.                        23
AJOD INSURANCE LIMITED, NEPAL               21
SANIMA INSURANCE COMPANY LIMITED            21
GENERAL INSURANCE COMPANY LIMITED, NEPAL    20
EVEREST INSURANCE CO.LTD                    19
RASTRIYA BEEMA SANSTHAN                     19
IME GENERAL INSURANCE COMPANY,NEPAL         17
PREMIER INSURANCE CO.(NEPAL) LTD.           11
Name: count, dtype: int64

In [79]:
# Create a list of values to filter, specifically anything related to CAT 
remove_list = ['XL-CAT THIRD','XL-CAT SECOND','XL-CAT FIRST','XL-CAT FOURTH','XL-CAT FIFTH','XL-RISK/CAT FOURTH','XL-RISK/CAT FIFTH','XL-CAT SIXTH']

# Remove rows based on the list of values
filtered_nepal_2019 = nepal_2019[nepal_2019['TYPE_OF_PARTICIPATION'].isin(remove_list)]
data = filtered_nepal_2019['TYPE_OF_PARTICIPATION']

x = filtered_nepal_2019['ACT_LOB'].value_counts()
print('The frequency of each LOB is as follows ', x)

The frequency of each LOB is as follows  ACT_LOB
FIRE             103
WHOLE_ACCOUNT     29
MOTOR             15
Name: count, dtype: int64


In [80]:
filtered_nepal_2019['Diff'].value_counts()
count_negative = (filtered_nepal_2019['Diff'] < 0).sum()
count_positive = (filtered_nepal_2019['Diff'] > 0).sum()
print('The number of negative difference values is: ',count_negative, 'and the number of positive values is: ',count_positive)
print('\n The are more positives than negatives')

The number of negative difference values is:  6 and the number of positive values is:  121

 The are more positives than negatives


### 2020

In [81]:
#CONTRACT_NO CEDENT_NAME
print('There are 315 unique contracts in 2020, from 17 cedants')
print('\n These are:', '\n\n\n The numbers on the right indicate the number of times the cedant name appears in the table')
current = nepal_2020[['CONTRACT_NO','CEDENT_NAME']].drop_duplicates()
current['CEDENT_NAME'].value_counts()

There are 315 unique contracts in 2020, from 17 cedants

 These are: 


 The numbers on the right indicate the number of times the cedant name appears in the table


CEDENT_NAME
SIDDHARTHA INSURANCE CO. LTD. NEPAL         38
RASTRIYA BEEMA SANSTHAN                     35
SANIMA INSURANCE COMPANY LIMITED            32
NECO INSURANCE CO. LTD                      27
NEPAL INSURANCE LTD.                        26
AJOD INSURANCE LIMITED, NEPAL               25
GENERAL INSURANCE COMPANY LIMITED, NEPAL    24
PRABHU INSURANCE LIMITED,NEPAL              24
PRUDENTIAL INSURANCE CO LTD                 24
EVEREST INSURANCE CO.LTD                    18
IME GENERAL INSURANCE COMPANY,NEPAL         17
PREMIER INSURANCE CO.(NEPAL) LTD.           13
HIMALAYAN GENERAL INSURANCE CO LTD          12
Name: count, dtype: int64

In [82]:
# Create a list of values to filter, specifically anything related to CAT 
remove_list = ['XL-CAT THIRD','XL-CAT SECOND','XL-CAT FIRST','XL-CAT FOURTH','XL-CAT FIFTH','XL-RISK/CAT FOURTH','XL-RISK/CAT FIFTH','XL-CAT SIXTH']

# Remove rows based on the list of values
filtered_nepal_2020 = nepal_2020[nepal_2020['TYPE_OF_PARTICIPATION'].isin(remove_list)]
data = filtered_nepal_2020['TYPE_OF_PARTICIPATION']
x = filtered_nepal_2020['ACT_LOB'].value_counts()
print('The frequency of each LOB is as follows ', x)

The frequency of each LOB is as follows  ACT_LOB
FIRE             73
WHOLE_ACCOUNT    39
MOTOR            12
Name: count, dtype: int64


In [83]:
filtered_nepal_2020['Diff'].value_counts()
count_negative = (filtered_nepal_2020['Diff'] < 0).sum()
count_positive = (filtered_nepal_2020['Diff'] > 0).sum()
print('The number of negative difference values is: ',count_negative, 'and the number of positive values is: ',count_positive)
print('\n The are more positives than negatives')



The number of negative difference values is:  9 and the number of positive values is:  98

 The are more positives than negatives


### 2021

In [84]:
#CONTRACT_NO CEDENT_NAME
print('There are 352 unique contracts in 2021, from 16 cedants')
print('\n These are:', '\n\n\n The numbers on the right indicate the number of times the cedant name appears in the table')
current = nepal_2021[['CONTRACT_NO','CEDENT_NAME']].drop_duplicates()
current['CEDENT_NAME'].value_counts()

There are 352 unique contracts in 2021, from 16 cedants

 These are: 


 The numbers on the right indicate the number of times the cedant name appears in the table


CEDENT_NAME
SIDDHARTHA INSURANCE CO. LTD. NEPAL         35
HIMALAYAN GENERAL INSURANCE CO LTD          35
AJOD INSURANCE LIMITED, NEPAL               34
GENERAL INSURANCE COMPANY LIMITED, NEPAL    34
PRUDENTIAL INSURANCE CO LTD                 32
PRABHU INSURANCE LIMITED,NEPAL              32
RASTRIYA BEEMA SANSTHAN                     28
NEPAL INSURANCE LTD.                        28
NECO INSURANCE CO. LTD                      28
EVEREST INSURANCE CO.LTD                    26
IME GENERAL INSURANCE COMPANY,NEPAL         26
PREMIER INSURANCE CO.(NEPAL) LTD.           14
Name: count, dtype: int64

In [85]:
# Create a list of values to filter, specifically anything related to CAT 
remove_list = ['XL-CAT THIRD','XL-CAT SECOND','XL-CAT FIRST','XL-CAT FOURTH','XL-CAT FIFTH','XL-RISK/CAT FOURTH','XL-RISK/CAT FIFTH','XL-CAT SIXTH']

# Remove rows based on the list of values
filtered_nepal_2021 = nepal_2021[nepal_2021['TYPE_OF_PARTICIPATION'].isin(remove_list)]
data = filtered_nepal_2021['TYPE_OF_PARTICIPATION']
x = filtered_nepal_2021['ACT_LOB'].value_counts()
print('The frequency of each LOB is as follows ', x)

The frequency of each LOB is as follows  ACT_LOB
FIRE             62
WHOLE_ACCOUNT    50
MOTOR             8
Name: count, dtype: int64


In [86]:
filtered_nepal_2021['Diff'].value_counts()
count_negative = (filtered_nepal_2021['Diff'] < 0).sum()
count_positive = (filtered_nepal_2021['Diff'] > 0).sum()
print('The number of negative difference values is: ',count_negative, 'and the number of positive values is: ',count_positive)
print('\n The are more positives than negatives')



The number of negative difference values is:  9 and the number of positive values is:  111

 The are more positives than negatives


### 2022

In [87]:
#CONTRACT_NO CEDENT_NAME
print('There are 150 unique contracts in 2022, from 11 cedants')
print('\n These are:', '\n\n\n The numbers on the right indicate the number of times the cedant name appears in the table')
current = nepal_2022[['CONTRACT_NO','CEDENT_NAME']].drop_duplicates()
current['CEDENT_NAME'].value_counts()

There are 150 unique contracts in 2022, from 11 cedants

 These are: 


 The numbers on the right indicate the number of times the cedant name appears in the table


CEDENT_NAME
HIMALAYAN GENERAL INSURANCE CO LTD          17
PRABHU INSURANCE LIMITED,NEPAL              16
PRUDENTIAL INSURANCE CO LTD                 15
RASTRIYA BEEMA SANSTHAN                     14
IME GENERAL INSURANCE COMPANY,NEPAL         14
AJOD INSURANCE LIMITED, NEPAL               14
GENERAL INSURANCE COMPANY LIMITED, NEPAL    14
SIDDHARTHA INSURANCE CO. LTD. NEPAL         13
NECO INSURANCE CO. LTD                      12
NEPAL INSURANCE LTD.                        11
PREMIER INSURANCE CO.(NEPAL) LTD.           10
Name: count, dtype: int64

In [88]:
x = filtered_nepal_2020['ACT_LOB'].value_counts()
print(' The frequency of each LOB is as follows ', x)

 The frequency of each LOB is as follows  ACT_LOB
FIRE             73
WHOLE_ACCOUNT    39
MOTOR            12
Name: count, dtype: int64


In [89]:
# Remove rows based on the list of values
filtered_nepal_2022 = nepal_2022[nepal_2022['TYPE_OF_PARTICIPATION'].isin(remove_list)]
filtered_nepal_2022 = nepal_2022.loc[nepal_2022['TYPE_OF_PARTICIPATION'].isin(['XL-CAT THIRD','XL-CAT SECOND','XL-CAT FIRST','XL-CAT FOURTH','XL-CAT FIFTH','XL-RISK/CAT FOURTH','XL-RISK/CAT FIFTH','XL-CAT SIXTH'])]

data = filtered_nepal_2022['TYPE_OF_PARTICIPATION']

# Compute value counts using pandas value_counts()
value_counts = data.value_counts()
filtered_nepal_2022

x = filtered_nepal_2022['ACT_LOB'].value_counts()
print('The frequency of each LOB is as follows ', x)  



The frequency of each LOB is as follows  Series([], Name: count, dtype: int64)


### There are no CAT related entries as of 2022